In [ ]:
from google.colab import drive
import pandas as pd
import random
!pip install openai
from openai import OpenAI

from openai import OpenAI
client = OpenAI(api_key=) #insert openai api key here

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')
reddit_dataset = '/content/drive/My Drive/continuity_error_updated_stories.csv'
df = pd.read_csv(reddit_dataset)

In [ ]:
with open(reddit_dataset, 'r') as file:
    stories = file.readlines()

In [ ]:
selected_stories = random.sample(stories, 1000)

In [ ]:
df = pd.DataFrame(selected_stories, columns=['story'])
df['error'] = None
df['position'] = None
df['response'] = None
df

In [ ]:
for index, row in df.iterrows():
    if pd.isnull(row['error']):
        prompt = "Pretend you were the author of the subsequent story. You want to insert one major continuity error, a lapse in the self-consistency of the narrative, somewhere in the story you've written (one sentence long), that a reader would only notice if they are paying close attention. Return two things: the sentence you'd add and the number of where you would insert it (count the number of '<nl>' that came before it in the story)."
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt + row['story']}
            ]
        )

        completion_text = response.choices[0].message.content
        if '#' in completion_text:
          error, position = completion_text.rsplit('#', 1)
          position = position.strip()
        else:
          continue

        df.at[index, 'error'] = completion_text
        df.at[index, 'position'] = position
        df.at[index, 'response'] = completion_text

        print (completion_text)

        df.to_csv('updated_stories.csv', index=False)

In [ ]:
# Function to strip quotes, strip white space, and add "<nl>" if not present
def clean_error_text(text):
    text = text.strip('"')
    text = text.strip()
    if not text.endswith('<nl>'):
        text += '<nl>'
    return text

df['error'] = df['error'].apply(clean_error_text)
df.to_csv('updated_stories.csv', index=False)
df['error']

Resuming, Data Cleaning/Processing

In [ ]:
drive.mount('/content/drive', force_remount=True)
reddit_dataset = '/content/drive/My Drive/updated_stories(7).csv'
df = pd.read_csv(reddit_dataset)

In [ ]:
def error_insert(row):
    position = int(row['position'])
    story_parts = row['story'].split('<nl>')
    insert_position = min(position, len(story_parts))
    story_parts.insert(insert_position, row['error'])
    return '<nl>'.join(story_parts)

df['modified'] = df.apply(error_insert, axis=1)

In [ ]:
df.to_csv('continuity_error_updated_stories.csv', index=False)

Split into Train/Dev/Test (80/10/10)

In [ ]:
from sklearn.model_selection import train_test_split

drive.mount('/content/drive', force_remount=True)
reddit_dataset = '/content/drive/My Drive/continuity_error_updated_stories.csv'
df = pd.read_csv(reddit_dataset)

df = df.sample(frac=1).reset_index(drop=True)

# Split the dataset
train_size = 0.8
validation_size = 0.1
test_size = 0.1

train_df, temp_df = train_test_split(df, train_size=train_size, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=test_size / (test_size + validation_size), random_state=42)

train_df.to_csv('train_dataset.csv', index=False)
validation_df.to_csv('validation_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

Mounted at /content/drive
